In [1]:
import numpy as np
import cv2

In [37]:
# check python 'data descriptors'. Necessary for overloading the variable video (int = 0 for webcam and string = video_name)
class OpticalFlowComputer:
    
    # Video captured: video = None
    
    # Mask image for drawing purposes: mask = None
    
    # Parameters for corner detection: feature_params = None
    
    # Parameters for Lucas-Kanade Optical Flow: lk_params = None
    
    # Create some random colors 
    color = None
    
    def __init__(self, video, 
                 maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7, 
                 winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)):
        self.video = video
        
        self.maxCorners = maxCorners 
        self.qualityLevel = qualityLevel 
        self.minDistance = minDistance 
        self.blockSize = blockSize
        self.setFeatureParams()
                    
        self.winSize = winSize
        self.maxLevel = maxLevel 
        self.criteria = criteria
        self.setLK_Params()
        
        OpticalFlowComputer.color = np.random.randint(0, 255, (100, 3))
    
    def __del__(self):
        self.video.release()
        print('Destructor called. VideoCapture released.')
    
    # Getter Methods
    def getVideoCapture(self):
        return self.video
        
    def getMaxCorners(self):
        return self.maxCorners
    
    def getQualityLevel(self):
        return self.qualityLevel
        
    def getMinDistance(self):
        return self.minDistance
        
    def getBlockSize(self):
        return self.blockSize
        
    def getWinSize(self):
        return self.winSize
        
    def getMaxLevel(self):
        return self.maxLevel
        
    def getCriteria(self):
        return self.criteria
        
    # Setter Methods
    def setVideoCapture(self, video):
        self.video = video
        
    def setMaxCorners(self, maxCorners = 100):
        self.maxCorners = maxCorners
        self.setFeatureParams()
    
    def setQualityLevel(self, qualityLevel = 0.3):
        self.qualityLevel = qualityLevel
        self.setFeatureParams()
        
    def setMinDistance(self, minDistance = 7):
        self.minDistance = minDistance
        self.setFeatureParams()
        
    def setBlockSize(self, blockSize = 7):
        self.blockSize = blockSize
        self.setFeatureParams()
        
    def setWinSize(self, winSize = (15, 15)):
        self.winSize = winSize
        self.setLK_Params()
        
    def setMaxLevel(self, maxLevel = 2):
        self.maxLevel = maxLevel
        self.setLK_Params()
        
    def setCriteria(self, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)):
        self.criteria = criteria
        self.setLK_Params()
        
    def setFeatureParams(self):
        self.feature_params = dict(maxCorners = self.maxCorners, 
                                   qualityLevel = self.qualityLevel, 
                                   minDistance = self.minDistance, 
                                   blockSize = self.blockSize)
        
    def setLK_Params(self):
        self.lk_params = dict(winSize = self.winSize, 
                              maxLevel = self.maxLevel, 
                              criteria = self.criteria)
        
    # Display Sparse Optical Flow
    def displaySparseOpticalFlow(self):
        # Take first frame and find corners in it 
        ret, old_frame = self.video.read() 
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY) 
        old_pts = cv2.goodFeaturesToTrack(old_gray, mask = None, **self.feature_params)
        
        # Create a mask image for drawing purposes 
        mask = np.zeros_like(old_frame)
        
        while(1):
    
            ret, frame = self.video.read()
            if ret == 1:
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
                # Calculate optical flow
                new_pts, statuses, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, old_pts, None, **self.lk_params)
        
                # Select good points
                good_new = new_pts[statuses == 1]
                good_old = old_pts[statuses == 1]

                # Draw the tracks
                for i, (new, old) in enumerate(zip(good_new, good_old)):
        
                    new_x, new_y = new.ravel()
                    old_x, old_y = old.ravel()
                    mask = cv2.line(mask, (new_x, new_y), (old_x, old_y), OpticalFlowComputer.color[i].tolist(), 2)
                    frame = cv2.circle(frame, (new_x, new_y), 5, OpticalFlowComputer.color[i].tolist(), -1)
        
                img = cv2.add(frame, mask)
                img = cv2.resize(img, (1280, 720))
                cv2.imshow('frame', img)
    
                k = cv2.waitKey(25)
                if k == 27:
                    break
        
                # Update previous frame and points
                old_gray = frame_gray.copy()
                old_pts = good_new.reshape(-1, 1, 2)
    
            else:
                break
                
        cv2.destroyAllWindows()

<h1>Experiment 1 - Standard Parameters</h1>

<h2>Video 1</h2>

In [38]:
cap = cv2.VideoCapture('dataset/BigBunnyWakesUp/BigBunnyWakesUp.mp4')
myOFComputer = OpticalFlowComputer(cap)

In [39]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 2</h2>

In [5]:
cap = cv2.VideoCapture('dataset/JumpingKid/JumpingKid.mp4')
myOFComputer.setVideoCapture(cap)

In [6]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 3</h2>

In [7]:
cap = cv2.VideoCapture('dataset/ClassicCar/ClassicCar.mp4')
myOFComputer.setVideoCapture(cap)

In [8]:
myOFComputer.displaySparseOpticalFlow()

<h2>Webcam</h2>

In [9]:
cap = cv2.VideoCapture(0)
myOFComputer.setVideoCapture(cap)

In [10]:
myOFComputer.displaySparseOpticalFlow()

In [11]:
cv2.destroyAllWindows()

<h1>Experiment 2 - Less Features Selected</h1>

<h2>Video 1</h2>

In [40]:
cap = cv2.VideoCapture('dataset/BigBunnyWakesUp/BigBunnyWakesUp.mp4')
myOFComputer.setVideoCapture(cap)
myOFComputer.setMaxCorners(maxCorners = 5)

In [41]:
myOFComputer.getMaxCorners()

5

In [42]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 2</h2>

In [43]:
cap = cv2.VideoCapture('dataset/JumpingKid/JumpingKid.mp4')
myOFComputer.setVideoCapture(cap)

In [44]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 3</h2>

In [45]:
cap = cv2.VideoCapture('dataset/ClassicCar/ClassicCar.mp4')
myOFComputer.setVideoCapture(cap)

In [46]:
myOFComputer.displaySparseOpticalFlow()

<h2>Webcam</h2>

In [47]:
cap = cv2.VideoCapture(0)
myOFComputer.setVideoCapture(cap)

In [48]:
myOFComputer.displaySparseOpticalFlow()

In [49]:
cv2.destroyAllWindows()

<h1>Experiment 3 - Up-Sampling Pyramid Max Level Increased</h1>

<h2>Video 1</h2>

In [55]:
cap = cv2.VideoCapture('dataset/BigBunnyWakesUp/BigBunnyWakesUp.mp4')
myOFComputer.setVideoCapture(cap)
myOFComputer.setMaxCorners(maxCorners = 100)
myOFComputer.setMaxLevel(maxLevel = 10)

In [56]:
myOFComputer.getMaxLevel()

10

In [57]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 2</h2>

In [58]:
cap = cv2.VideoCapture('dataset/JumpingKid/JumpingKid.mp4')
myOFComputer.setVideoCapture(cap)

In [59]:
myOFComputer.displaySparseOpticalFlow()

<h2>Video 3</h2>

In [60]:
cap = cv2.VideoCapture('dataset/ClassicCar/ClassicCar.mp4')
myOFComputer.setVideoCapture(cap)

In [61]:
myOFComputer.displaySparseOpticalFlow()

<h2>Webcam</h2>

In [62]:
cap = cv2.VideoCapture(0)
myOFComputer.setVideoCapture(cap)

In [63]:
myOFComputer.displaySparseOpticalFlow()

TypeError: 'NoneType' object is not subscriptable

In [64]:
cv2.destroyAllWindows()

In [65]:
del myOFComputer

In [66]:
cap.release()